In [ ]:
!pip install huggingface-hub
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from datasets import load_dataset, DatasetDict
from random import randrange
test_dataset_path = '/content/drive/My Drive/test_dataset.csv'
test_dataset = load_dataset("csv", data_files=test_dataset_path, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a sample in the fake news classification dataset.

    :param sample: A dictionary representing a row from the fake news classification dataset.
    """
    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    #RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}"  # Assuming 'text' is the column with the news content
    #response = f"{RESPONSE_KEY}\n{sample['output']}"  # Assuming 'target' is the column with the label 'fake' or 'real'
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample


In [ ]:
test_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 8600
})

In [ ]:
def preprocess_text(dataset: str):

    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)


    return dataset

In [ ]:
formatted_dataset=preprocess_text(test_dataset)

Preprocessing dataset...


Map:   0%|          | 0/8600 [00:00<?, ? examples/s]

In [ ]:
formatted_dataset=formatted_dataset.map(remove_columns=["instruction", "input"])

Map:   0%|          | 0/8600 [00:00<?, ? examples/s]

In [ ]:
formatted_dataset

Dataset({
    features: ['output', 'text'],
    num_rows: 8600
})

In [ ]:
from huggingface_hub.hf_api import HfFolder
#from langchain import HuggingFacePipeline


from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch




# The HF model is being loaded from a saved state on the disk
HfFolder.save_token("xxxx")
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("jua2020/fakenews-binaryclassificationV2-llama-2-7b")
model = AutoModelForCausalLM.from_pretrained("jua2020/fakenews-binaryclassificationV2-llama-2-7b")


# ... (Rest of the code is cut off)


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
# Take 10 random samples from the dataset
samples = formatted_dataset.shuffle(seed=42).select(range(10))
# Prepare to collect the predictions and actual labels
predictions = []
actual_labels = []

# Generate predictions
for i, sample in enumerate(samples):
    # Tokenize the text from the 'text' column of the sample
    input_ids = tokenizer(sample['text'], return_tensors="pt").input_ids

    # Generate the output using the model
    outputs = model.generate(input_ids, max_new_tokens=200)

    # Decode the generated token IDs to text
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Here you need to convert 'decoded_output' to the same format as your 'output' column.
    # This step highly depends on what the 'output' looks like.
    # For example, if your output is simply '0' or '1', you might do:
    prediction = int(decoded_output.strip() == '1')
    predictions.append(prediction)

    # Store the actual label
    actual_labels.append(sample['output'])

# Now, calculate the accuracy
accuracy = sum(torch.tensor(predictions) == torch.tensor(actual_labels)) / len(actual_labels)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6000000238418579


In [ ]:
#input_ids=tokenizer(prompt,return_tensors="pt").input_ids
#outputs=model.generate(input_ids,max_new_tokens=200)

In [ ]:
#decoded_output=tokenizer.decode(outputs[0])

#if '0' in decoded_output:
    #final_output='0'
#elif '1' in decoded_output:
    # final_output='1'
#else:
  #final_output='Not clear'


#print(final_output)


